In [1]:
import datetime 
#from gensim.models import Word2Vec 
from itertools import combinations 
import random 
import pandas as pd 
from sklearn.calibration import label_binarize 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, f1_score, roc_auc_score 
from sklearn.decomposition import LatentDirichletAllocation 
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.manifold import TSNE 
import numpy as np 
import re 
import string 
from transformers import BertTokenizer, BertModel 
#import shap 
import matplotlib.pyplot as plt 
from imblearn.over_sampling import SMOTE 
import seaborn as sns 
from ast import literal_eval
#from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_predict
from langdetect import detect
import pickle

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
import pandas as pd
import numpy as np
import spacy
import re
import string
from gensim.models import Word2Vec
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from imblearn.over_sampling import SMOTE

In [3]:
comments_df = pd.read_csv('comments_f2v_fts.csv')
videos_df = pd.read_csv('vid_fin.csv')

In [4]:
comments_df = comments_df[:1000]

In [5]:
# Функция для подсчета иностранных слов
def count_foreign_words(text):
    if isinstance(text, str):
        foreign_word_count = 0
        words = text.split()
        for word in words:
            try:
                lang = detect(word)
                if lang != 'ru':
                    foreign_word_count += 1
            except LangDetectException:
                continue  # Пропускаем слово, если не удалось определить язык
        return foreign_word_count
    return 0

In [6]:
comments_df['Foreign_Words'] = comments_df['Processed_Text'].apply(count_foreign_words)

In [5]:

from deeppavlov import build_model, configs

model = build_model('levenshtein_corrector_ru', download=True)

2024-05-25 07:14:46.348 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz download because of matching hashes
2024-05-25 07:14:46.962 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz download because of matching hashes


In [49]:
i=899
print(comments_df['Processed_Text'][i])
model([comments_df['Processed_Text'][i]])

очередной дрянь


['очередной дрянь']

In [7]:
import pandas as pd
import spacy
import re
import string
from gensim.models import Word2Vec
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")
comments_df['Text'] = comments_df['Text'].fillna('').astype(str)
# Функция для предобработки текста
def preprocess_text(text):
    # Обработка NaN значений
    if pd.isnull(text):
        return ""
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление ссылок и других ненужных символов
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Токенизация и лемматизация
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and token.is_alpha]
    return ' '.join(tokens)

# Применение предобработки к каждому тексту
comments_df['Processed_Text'] = comments_df['Text'].apply(preprocess_text)

# Создание корпуса из текстов комментариев
corpus = comments_df['Processed_Text'].tolist()

# Разделение текста на предложения
sentences = [text.split() for text in corpus]

# Создание модели Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_comment_vector(comment, model):
    words = comment.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Загрузка предобученной модели Word2Vec
#with open('ruwiki_20180420_300d.txt', 'rb') as f:
#    pretrained_model = pickle.load(f)    

#pretrained_model = Word2Vec.load_word2vec_format('model.bin', binary=False)

# Преобразование всех комментариев в векторы и добавление в DataFrame
#comments_df['Vector_PT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, pretrained_model))



# Дообучение предобученной модели на ваших данных
#pretrained_model.build_vocab(sentences, update=True)
#pretrained_model.train(sentences, total_examples=pretrained_model.corpus_count, epochs=pretrained_model.epochs)
#comments_df['Vector_PT_FT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, pretrained_model))
# Функция для получения вектора комментария

    
def extract_additional_features(comment):
    length = len(comment)
    uppercase_count = sum(1 for c in comment if c.isupper())
    punctuation_count = sum(1 for c in comment if c in string.punctuation)
    emoji_count = len(re.findall(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+', comment))
    return np.array([length, uppercase_count, punctuation_count, emoji_count])

# Преобразование всех комментариев в векторы и добавление дополнительных признаков
def get_combined_vector(comment, model):
    word_vector = get_comment_vector(comment, model)
    additional_features = extract_additional_features(comment)
    combined_vector = np.concatenate((word_vector, additional_features), axis=None)
    return combined_vector

# Пример получения вектора для одного комментария
example_comment = comments_df['Processed_Text'].iloc[0]
comment_vector = get_comment_vector(example_comment, word2vec_model)
print(comment_vector)

# Преобразование всех комментариев в векторы
comments_df['Vector'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, word2vec_model))

print(comments_df[['Text', 'Processed_Text', 'Vector']].head())


[-1.0654504e-03  1.0320158e-03  2.7321533e-03  1.4133225e-03
 -4.2301468e-03 -1.9832016e-03 -2.9613604e-03 -6.0447521e-04
 -8.1446749e-05  1.2731079e-03 -6.1814737e-04 -1.8282729e-03
  3.4352015e-03 -1.1375772e-03 -1.5653613e-03 -9.2165184e-04
  4.9283486e-03 -3.8856727e-03 -2.8179768e-03  2.2492285e-03
  1.7177633e-03  2.8284639e-04  1.8656129e-03 -8.8580657e-04
 -1.1668840e-04 -1.0964335e-03  2.6085475e-04  1.4139909e-03
 -3.5522154e-03  4.7996466e-04 -5.5389036e-04  4.3803913e-04
  6.9834321e-04  7.9575204e-04 -9.9613098e-06 -4.5002007e-04
 -1.0592618e-03 -1.8349282e-03  3.6572823e-03 -1.5381719e-03
  3.3216353e-04  1.5853625e-04  9.0166635e-04  4.2739804e-03
  1.3446755e-03 -1.2062155e-03  1.8733098e-04  2.1990722e-04
 -1.2826540e-03  1.5358465e-03 -5.7199667e-04  7.5791695e-04
 -9.7678741e-04  1.3685594e-03 -2.9003841e-04  5.1651604e-04
 -3.3517629e-03  1.0955312e-03  2.8434812e-04 -4.6028954e-04
 -2.3505653e-03  9.0783462e-04 -2.7266983e-03 -3.7410349e-04
 -2.3560040e-04 -1.16132

In [ ]:
import pandas as pd
import spacy
import re
import string
import numpy as np
from gensim.models import FastText
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words
import fasttext

# Load pre-trained FastText model for Russian
model_ft = fasttext.load_model('cc.ru.300.bin')

# Создание модели FastText (новая модель, обученная на ваших данных)
#fasttext_model = FastText(sentences, vector_size=300, window=5, min_count=1, workers=4)

# Загрузка предобученной модели FastText
#model_path = 'cc.ru.300.bin'  # замените на путь к вашей скачанной модели
#pretrained_model = FastText.load_facebook_model(model_path)
#pretrained_model = FastTexload_facebook_vectors(model_path)
# Дообучение предобученной модели на ваших данных
#pretrained_model.build_vocab(sentences, update=True)
#pretrained_model.train(sentences, total_examples=pretrained_model.corpus_count, epochs=pretrained_model.epochs)

# Функция для получения вектора комментария
def get_comment_vector(comment, model):
    if comment:
        print(len(model.get_sentence_vector(comment)))
        return model.get_sentence_vector(comment)
    else:
        return np.zeros(model.vector_size)

# Преобразование всех комментариев в векторы и добавление в DataFrame
comments_df['Vector_FT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, model_ft))
#comments_df['Vector_PT_FT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, pretrained_model))

# Функция для получения дополнительных признаков
def extract_additional_features(comment):
    length = len(comment)
    uppercase_count = sum(1 for c in comment if c.isupper())
    punctuation_count = sum(1 for c in comment if c in string.punctuation)
    emoji_count = len(re.findall(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+', comment))
    return np.array([length, uppercase_count, punctuation_count, emoji_count])


# Вывод результатов
#print(comments_df[['Text', 'Processed_Text', 'Vector_PT']])


In [27]:
#comments_df.to_csv('vid_w2v.csv', index=False)

In [23]:
comments_df['Vector_features'] = comments_df['Text'].apply(lambda x: get_combined_vector(preprocess_text(x), word2vec_model))

In [ ]:
#comments_df['Vector'] = comments_df['Vector'].apply(lambda x: np.array(literal_eval(str(x))) if isinstance(x, str) else x)

In [40]:
#comments_df.to_csv('comments_f2v_fts.csv', index=False)

In [60]:
comm_features_label = pd.read_csv('vid_23.csv')

In [52]:
pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 1.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=a3e684b04efafa0fb29eee8d88f78b0dc880edaa2a2bc4e59bac53ad2103cd3e
  Stored in directory: /Users/pratappokkharel/Library/Caches/pip/wheels/5e/90/99/807a5ad861ce5d22c3c299a11df8cba9f31524f23ae6e645cb
Successfully built autocorrect
Note: you may need to restart the kernel to use updated packages.


In [34]:
from autocorrect import Speller
spell = Speller('ru')
i=100
print(comments_df['Processed_Text'][i])
spell(comments_df['Processed_Text'][i])


смотреть интервью воротить россиянин типо против война замечать емоций вражда поколение


'смотреть интервью воротить россиянин типо против война замечать эмоций вражда поколение'

In [64]:
import nltk
from nltk.corpus import brown

# Загружаем корпус слов из библиотеки NLTK
nltk.download('brown')
nltk.download('words')
words = set(nltk.corpus.words.words())
brown_words = nltk.FreqDist(w.lower() for w in brown.words())

def is_valid_word(word):
    return word in words

def split_word(word):
    max_prob = 0
    best_split = None
    for i in range(1, len(word)):
        word1, word2 = word[:i], word[i:]
        if is_valid_word(word1) and is_valid_word(word2):
            # Используем частоты из корпуса для оценки вероятности
            prob = brown_words[word1] * brown_words[word2]
            if prob > max_prob:
                max_prob = prob
                best_split = (word1, word2)
    return best_split if best_split else (word,)

# Примеры неправильных слов
incorrect_words = ['мамарама', 'книжкастол', 'счастьелюдей']

# Поиск и печать разделённых слов
for word in incorrect_words:
    split = split_word(word)
    print(f"{word} -> {' '.join(split)}")


[nltk_data] Downloading package brown to
[nltk_data]     /Users/pratappokkharel/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package words to
[nltk_data]     /Users/pratappokkharel/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


мамарама -> мамарама
книжкастол -> книжкастол
счастьелюдей -> счастьелюдей


In [50]:
comments_df['Processed_Text'][15]

'человек бываютвитает облако неплохоно понимать реальность тамнельзя переехать быстро включиться'

In [28]:
pip install pymorphy2

  Obtaining dependency information for pymorphy2 from https://files.pythonhosted.org/packages/07/57/b2ff2fae3376d4f3c697b9886b64a54b476e1a332c67eee9f88e7f1ae8c9/pymorphy2-0.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for pymorphy2-dicts-ru<3.0,>=2.4 from https://files.pythonhosted.org/packages/3a/79/bea0021eeb7eeefde22ef9e96badf174068a2dd20264b9a378f2be1cdd9e/pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl.metadata
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=2c48a3383c8984ab9c30828450e53fbbc2c6cbea3ee66563a15db49f13f6fc70
  Stored in directory: /Users/pratappokkharel/Library/Caches/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
Note: you may need

In [35]:
import nltk
import pymorphy2
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams


# Загружаем слова
with open('100000-russian-words.txt', 'r', encoding='utf-8') as f:
    correct_words = f.read().split()

# Примеры неправильных слов
incorrect_words = ['счасттливый', 'удевительное', 'интеллегентный']

# Функция для исправления слов
def correct_spelling(word, correct_words):
    morph = pymorphy2.MorphAnalyzer()
    temp = [(jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))), w) 
            for w in correct_words if w[0] == word[0]]
    return sorted(temp, key=lambda val: val[0])[0][1] if temp else word

# Поиск и печать исправленных слов
for word in incorrect_words:
    corrected_word = correct_spelling(word, correct_words)
    print(corrected_word)

    print(corrected_word)



AttributeError: module 'inspect' has no attribute 'getargspec'

In [64]:
comm_features_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1000 non-null   int64  
 1   author_name               999 non-null    object 
 2   text                      1000 non-null   object 
 3   reply_count               1000 non-null   float64
 4   top_level                 1000 non-null   int64  
 5   index                     1000 non-null   int64  
 6   publishedAt               1000 non-null   object 
 7   updateAt                  1000 non-null   object 
 8   likeCount                 1000 non-null   float64
 9   Video_ID                  1000 non-null   int64  
 10  Label                     1000 non-null   object 
 11  Text                      995 non-null    object 
 12  Length                    1000 non-null   int64  
 13  UpperCase                 1000 non-null   int64  
 14  Punctuati

In [61]:
comments_df_fin = comments_df.iloc[:1000]

In [62]:
comments_df_fin['Label'] = comm_features_label['Label']

/var/folders/00/4_gxgzyx7131tsyx_ymz0hsm0000gn/T/ipykernel_1945/3002675891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df_fin['Label'] = comm_features_label['Label']


# Word2vec

In [66]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train, y_train)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.92
F1 Score (Micro): 0.92
F1 Score (Macro): 0.46208112874779544
ROC AUC (OvO): 0.6558157219469972
ROC AUC (OvR): 0.6558157219469972
              precision    recall  f1-score   support

         bot       1.00      0.27      0.43        11
      normal       0.92      1.00      0.96       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.92       200
   macro avg       0.64      0.42      0.46       200
weighted avg       0.89      0.92      0.89       200


Logistic Regression:
Accuracy: 0.905
F1 Score (Micro): 0.905
F1 Score (Macro): 0.31671041119860016
ROC AUC (OvO): 0.6053368468977264
ROC AUC (OvR): 0.6053368468977264
              precision    recall  f1-score   support

         bot       0.00      0.00      0.00        11
      normal       0.91      1.00      0.95       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.91       200
   macro avg       0

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

# Word2vec + SMOTE

In [67]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.795
F1 Score (Micro): 0.795
F1 Score (Macro): 0.39221304567672344
ROC AUC (OvO): 0.7197904421437938
ROC AUC (OvR): 0.7197904421437938
              precision    recall  f1-score   support

         bot       0.22      0.45      0.29        11
      normal       0.92      0.85      0.88       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.80       200
   macro avg       0.38      0.44      0.39       200
weighted avg       0.84      0.80      0.81       200


Logistic Regression:
Accuracy: 0.275
F1 Score (Micro): 0.275
F1 Score (Macro): 0.23045065220807792
ROC AUC (OvO): 0.6539688589267714
ROC AUC (OvR): 0.6539688589267714
              precision    recall  f1-score   support

         bot       0.08      0.64      0.14        11
      normal       0.95      0.23      0.37       181
       troll       0.10      0.88      0.19         8

    accuracy                           0.28       200
   macro avg      

# Standardized Word2vec + SMOTE + 

In [68]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Стандартизация признаков
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test = scaler.transform(X_test)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.79
F1 Score (Micro): 0.79
F1 Score (Macro): 0.3883415435139573
ROC AUC (OvO): 0.7183169549747657
ROC AUC (OvR): 0.7183169549747657
              precision    recall  f1-score   support

         bot       0.21      0.45      0.29        11
      normal       0.92      0.85      0.88       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.79       200
   macro avg       0.37      0.43      0.39       200
weighted avg       0.84      0.79      0.81       200


Logistic Regression:
Accuracy: 0.52
F1 Score (Micro): 0.52
F1 Score (Macro): 0.34145880574452003
ROC AUC (OvO): 0.6687594327671461
ROC AUC (OvR): 0.6687594327671461
              precision    recall  f1-score   support

         bot       0.11      0.64      0.19        11
      normal       0.95      0.51      0.67       181
       troll       0.10      0.50      0.16         8

    accuracy                           0.52       200
   macro avg       0.39

# Word2vec + SMOTE + Features

In [69]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.64
F1 Score (Micro): 0.64
F1 Score (Macro): 0.36143237605691514
ROC AUC (OvO): 0.686852599321098
ROC AUC (OvR): 0.686852599321098
              precision    recall  f1-score   support

         bot       0.14      0.45      0.21        11
      normal       0.91      0.67      0.77       181
       troll       0.07      0.25      0.11         8

    accuracy                           0.64       200
   macro avg       0.37      0.46      0.36       200
weighted avg       0.83      0.64      0.71       200


Logistic Regression:
Accuracy: 0.26
F1 Score (Micro): 0.26
F1 Score (Macro): 0.22210820446114565
ROC AUC (OvO): 0.6332451479631573
ROC AUC (OvR): 0.6332451479631573
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       0.95      0.22      0.35       181
       troll       0.12      0.88      0.21         8

    accuracy                           0.26       200
   macro avg       0.38 

# Word2vec + SMOTE + Standardized Features


In [70]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# Выделение последних четырех признаков для стандартизации
X_additional = X[:, -4:]

# Стандартизация последних четырех признаков
scaler = StandardScaler()
X_additional_scaled = scaler.fit_transform(X_additional)

# Объединение стандартизированных признаков с остальными
X_combined = np.hstack((X[:, :-4], X_additional_scaled))

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.775
F1 Score (Micro): 0.775
F1 Score (Macro): 0.44004121423476267
ROC AUC (OvO): 0.6904555357813633
ROC AUC (OvR): 0.6904555357813633
              precision    recall  f1-score   support

         bot       0.19      0.45      0.27        11
      normal       0.93      0.82      0.87       181
       troll       0.14      0.25      0.18         8

    accuracy                           0.78       200
   macro avg       0.42      0.51      0.44       200
weighted avg       0.85      0.78      0.81       200


Logistic Regression:
Accuracy: 0.25
F1 Score (Micro): 0.25
F1 Score (Macro): 0.2155307994757536
ROC AUC (OvO): 0.6245253183651656
ROC AUC (OvR): 0.6245253183651656
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       1.00      0.20      0.34       181
       troll       0.11      0.88      0.20         8

    accuracy                           0.25       200
   macro avg       0.

# PCA (Word2vec) + SMOTE

In [73]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.decomposition import PCA

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# Применение PCA для уменьшения размерности векторов Word2Vec
pca = PCA(n_components=25, random_state=42)
X_pca = pca.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42, kernel='rbf')
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.865
F1 Score (Micro): 0.865
F1 Score (Macro): 0.5404007756948933
ROC AUC (OvO): 0.8290817200955655
ROC AUC (OvR): 0.8290817200955655
              precision    recall  f1-score   support

         bot       0.40      0.55      0.46        11
      normal       0.94      0.91      0.92       181
       troll       0.22      0.25      0.24         8

    accuracy                           0.86       200
   macro avg       0.52      0.57      0.54       200
weighted avg       0.88      0.86      0.87       200


Logistic Regression:
Accuracy: 0.29
F1 Score (Micro): 0.29
F1 Score (Macro): 0.23942284294707203
ROC AUC (OvO): 0.6525923527472473
ROC AUC (OvR): 0.6525923527472473
              precision    recall  f1-score   support

         bot       0.08      0.64      0.14        11
      normal       0.96      0.24      0.39       181
       troll       0.11      0.88      0.19         8

    accuracy                           0.29       200
   macro avg       0.

In [40]:
# PCA (Standardized Word2vec) + SMOTE

In [60]:
comments_df_fin['Vector_features'][0].size

104

# PCA (Word2vec) + SMOTE + Features

In [74]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# Выделение последних четырех признаков для стандартизации
X_additional = X[:, -4:]

# Применение PCA для уменьшения размерности векторов Word2Vec
pca = PCA(n_components=25, random_state=42)
X_pca = pca.fit_transform(X[:, :-4])

# Стандартизация последних четырех признаков
#scaler = StandardScaler()
#X_additional_scaled = scaler.fit_transform(X_additional)

# Объединение стандартизированных признаков с остальными
X_combined = np.hstack((X_pca, X_additional))

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Создание и обучение моделей
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# Обучение моделей
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# Предсказание на тестовых данных и оценка моделей
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Вывод результатов для каждой модели
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.875
F1 Score (Micro): 0.875
F1 Score (Macro): 0.5481664517977367
ROC AUC (OvO): 0.7794330055545049
ROC AUC (OvR): 0.7794330055545049
              precision    recall  f1-score   support

         bot       0.40      0.55      0.46        11
      normal       0.94      0.92      0.93       181
       troll       0.25      0.25      0.25         8

    accuracy                           0.88       200
   macro avg       0.53      0.57      0.55       200
weighted avg       0.89      0.88      0.88       200


Logistic Regression:
Accuracy: 0.265
F1 Score (Micro): 0.265
F1 Score (Macro): 0.22559684253011336
ROC AUC (OvO): 0.634764792540074
ROC AUC (OvR): 0.634764792540074
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       0.95      0.22      0.36       181
       troll       0.12      0.88      0.21         8

    accuracy                           0.27       200
   macro avg       0.

In [58]:
# Разделение данных на признаки и целевую переменную
X = np.vstack(comments_df_fin['Vector_features'].values)
X_word2vec = X[:, :-4]
# Здесь можно добавить свои признаки
# Выделение последних четырех признаков для стандартизации
X_additional = X[:, -4:]
y = comments_df_fin['Label']

# Стандартизация данных
#scaler_word2vec = StandardScaler()
#X_word2vec = scaler_word2vec.fit_transform(X_word2vec)

scaler_additional = StandardScaler()
X_additional = scaler_additional.fit_transform(X_additional)

n_components = 25
# Применение PCA с выбранным числом компонентов
pca = PCA(n_components=n_components, random_state=42)
X_word2vec_pca = pca.fit_transform(X_word2vec)

# Разделение на обучающую и тестовую выборки
X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec_pca, y, test_size=0.2, random_state=42)
X_train_additional, X_test_additional, _, _ = train_test_split(X_additional, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке
smote = SMOTE(random_state=42)
X_train_word2vec_smote, y_train_smote = smote.fit_resample(X_train_word2vec, y_train)
X_train_additional_smote, _ = smote.fit_resample(X_train_additional, y_train)

# Создание моделей для каждой группы признаков
rf_model_word2vec = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model_word2vec = LogisticRegression(max_iter=1000, random_state=42)
gb_model_word2vec = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model_word2vec = SVC(probability=True, random_state=42)

rf_model_additional = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model_additional = LogisticRegression(max_iter=1000, random_state=42)
gb_model_additional = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model_additional = SVC(probability=True, random_state=42)

# Создание словаря базовых моделей
base_models_word2vec = {
    "Random Forest Word2Vec": rf_model_word2vec,
    #"Logistic Regression Word2Vec": lr_model_word2vec,
    "Gradient Boosting Word2Vec": gb_model_word2vec,
    #"SVC Word2Vec": svc_model_word2vec
}

base_models_additional = {
    #"Random Forest Additional": rf_model_additional,
    #"Logistic Regression Additional": lr_model_additional,
    #"Gradient Boosting Additional": gb_model_additional,
    "SVC Additional": svc_model_additional
}

# Получение метапризнаков через кросс-валидацию
def get_meta_features(model_dict, X, y):
    meta_features = np.zeros((X.shape[0], len(model_dict)))
    for i, (name, model) in enumerate(model_dict.items()):
        meta_features[:, i] = cross_val_predict(model, X, y, cv=5, method='predict_proba')[:, 1]
    return meta_features

meta_features_word2vec = get_meta_features(base_models_word2vec, X_word2vec_pca, y)
meta_features_additional = get_meta_features(base_models_additional, X_additional, y)

# Объединение метапризнаков
meta_features_combined = np.hstack([meta_features_word2vec, meta_features_additional])

# Разделение на обучающую и тестовую выборки для метамодели
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(meta_features_combined, y, test_size=0.2, random_state=42)

# Применение SMOTE к обучающей выборке метамодели
X_train_meta_smote, y_train_meta_smote = smote.fit_resample(X_train_meta, y_train_meta)

from sklearn.ensemble import GradientBoostingClassifier

# Обучение Logistic Regression как метамодели
lr_meta_model = LogisticRegression(max_iter=1000, random_state=42)
lr_meta_model.fit(X_train_meta_smote, y_train_meta_smote)

# Обучение Gradient Boosting как метамодели
gb_meta_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_meta_model.fit(X_train_meta, y_train_meta)

# Предсказание на тестовых данных и оценка Logistic Regression метамодели
y_pred_lr_meta = lr_meta_model.predict(X_test_meta)
y_proba_lr_meta = lr_meta_model.predict_proba(X_test_meta)

# Предсказание на тестовых данных и оценка Gradient Boosting метамодели
y_pred_gb_meta = gb_meta_model.predict(X_test_meta)
y_proba_gb_meta = gb_meta_model.predict_proba(X_test_meta)

# Функция для оценки модели
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# Оценка метамоделей
evaluate_model(y_test_meta, y_pred_lr_meta, y_proba_lr_meta, "Logistic Regression Meta Model")
evaluate_model(y_test_meta, y_pred_gb_meta, y_proba_gb_meta, "Gradient Boosting Meta Model")


Logistic Regression Meta Model:
Accuracy: 0.2830188679245283
F1 Score (Micro): 0.2830188679245283
F1 Score (Macro): 0.23836657169990505
ROC AUC (OvO): 0.44903846153846155
ROC AUC (OvR): 0.44903846153846155
              precision    recall  f1-score   support

         bot       0.10      0.40      0.15         5
      normal       0.79      0.28      0.41        40
       troll       0.11      0.25      0.15         8

    accuracy                           0.28        53
   macro avg       0.33      0.31      0.24        53
weighted avg       0.62      0.28      0.35        53


Gradient Boosting Meta Model:
Accuracy: 0.6981132075471698
F1 Score (Micro): 0.6981132075471698
F1 Score (Macro): 0.4102067183462533
ROC AUC (OvO): 0.491951566951567
ROC AUC (OvR): 0.491951566951567
              precision    recall  f1-score   support

         bot       0.33      0.20      0.25         5
      normal       0.76      0.88      0.81        40
       troll       0.25      0.12      0.17       

In [23]:
def preprocess_text(text):
    text = text.lower()  # Приведение текста к нижнему регистру
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Удаление ссылок
    text = re.sub(r'\@\w+|\#', '', text)  # Удаление упоминаний и хэштегов
    text = re.sub(r'\d+', '', text)  # Удаление цифр
    text = text.translate(str.maketrans('', '', string.punctuation))  # Удаление пунктуации
    return text

In [25]:
# Создание модели Word2Vec 
# Объединение текстов комментариев и видео в единый корпус 
corpus = comments_df['Text'].apply(preprocess_text).tolist() #+ videos_df['Video_Text'].apply(preprocess_text).tolist() 
print(corpus)
sentences = [text.split() for text in corpus] 
 
# Создание модели Word2Vec 
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4) 
 
# Функция для получения вектора комментария 
def get_comment_vector(comment, model): 
    words = comment.split() 
    word_vectors = [model.wv[word] for word in words if word in model.wv] 
    if len(word_vectors) == 0: 
        return np.zeros(model.vector_size) 
    else: 
        return np.mean(word_vectors, axis=0)

['классическая инфантильная тп придумала себе головняк на ровном месте умотала туда где теплее', 'зашквар', 'нарежьте видео со всеми ее моканиями плиз', 'если она еще раз чмокнет я оглохну навсегда\r\nбудто помидоры квашеные есть', 'smm jump  лучшая накрутка в интернете', 'инструкция яшина о том как квартиру приготовить к обыску слезть с собчак и надеть трусы', 'про корею выпуск великолепный как раньше сейчас опять пошла нудятина с бесконечными стендаперами и непонятными актрисками также пишутся феминитивы\r\nне понимаю почему об этом ни кто не пишет в комментах', 'да блятькто это юра и почему так важно мнение этих либерастных псевдо геополитиков', 'не очень умная женщинамного противоречий в ее расказеактриса одной ролиуехала это ее выбор', 'юра кто эти люди когда будут нормальные гости не интересно', 'ну ось маю надію що служба сбу україни візьме фрагмент інтервю до уваги так як вона свідчить дуже важливі факти про свого вітчима який виконує злочинні накази ', 'давай бурунов  urgent u

NameError: name 'Word2Vec' is not defined

In [16]:
import spacy
from gensim.utils import simple_preprocess
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

def lemma(word):
    for w in nlp(word):
        return w.lemma_

def process_words(texts):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts_out = []
    for sent in texts:
        texts_out.append([lemma(token) for token in sent])
    # remove stopwords after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

def preprocess(comment):
    # Tokenize and lemmatize
    tokens = [token.lemma_ for token in nlp(comment)]
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens).replace('\n', ' ')

# Пример использования
texts = [
    "Это пример текста для обработки. Включает стоп-слова и различные формы слов.",
    "Еще один пример текста, который будет лемматизирован и очищен от стоп-слов."
]

# Пример предобработки текстов
processed_texts = process_words(texts)
print(processed_texts)

# Пример предобработки одного комментария
comment = "Это пример комментария для лемматизации и удаления стоп-слов."
processed_comment = preprocess(comment)
print(processed_comment)

[['пример', 'текст', 'обработка', 'включать', 'стоп', 'слово', 'различный', 'форма', 'слово'], ['пример', 'текст', 'лемматизирован', 'очистить', 'стоп', 'слово']]
пример комментарий лемматизация удаление стопа - слово .


In [ ]:
import pandas as pd
import spacy
from gensim.utils import simple_preprocess
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Пример текста для обработки
texts = ['а мы это кто ты и твоя шиза🙂', 'мы кроме ураноса', 'это кто', 'ойкак хорошо на западено вся беда в самой и вижу слабого рядом мужчину', ...]

# Функция лемматизации и предобработки текста
def preprocess_text(text):
    # Обработка NaN значений
    if pd.isnull(text):
        return ""
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление ссылок и других ненужных символов
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Токенизация и лемматизация
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and token.is_alpha]
    return ' '.join(tokens)

# Применение предобработки к каждому тексту
processed_texts = [preprocess_text(text) for text in texts]

print(processed_texts)

# Создание корпуса из текстов комментариев
corpus = processed_texts

# Разделение текста на предложения
sentences = [text.split() for text in corpus]

# Создание модели Word2Vec
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Функция для получения вектора комментария
def get_comment_vector(comment, model):
    words = comment.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Пример получения вектора для одного комментария
comment = "Пример комментария для векторизации"
preprocessed_comment = preprocess_text(comment)
comment_vector = get_comment_vector(preprocessed_comment, word2vec_model)
print(comment_vector)
